# CS345 Project

For our dataset, we will be using the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/). Due to the size of this dataset, it could not be uploaded to github. Please download the dataset yourself, extract it, and move it to the "data" directory.

### Imports

In [ ]:
import os
import pandas as pd
import re
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

### Load the data

In [ ]:
def load_data(data_dir):
    data = []
    for sentiment in ['pos', 'neg']:
        sentiment_dir = os.path.join(data_dir, sentiment)
        print(f"Processing '{sentiment}' reviews...")
        file_list = os.listdir(sentiment_dir)
        # Use tqdm to create a progress bar as loading can take a while, we want to make sure it isn't hanging
        for filename in tqdm(file_list, desc=f"Loading {sentiment} files"):
            if filename.endswith('.txt'):
                filepath = os.path.join(sentiment_dir, filename)
                with open(filepath, 'r', encoding='utf-8') as f:
                    review = f.read()
                data.append({
                    'review': review,
                    'sentiment': 1 if sentiment == 'pos' else 0,
                })

    df = pd.DataFrame(data)
    print(f"Loaded {len(df)} reviews from '{data_dir}'")
    return df

train_data = load_data('data/aclImdb/train')
test_data = load_data('data/aclImdb/test')

### Let's check out some of the data

In [ ]:
print(train_data.head())

### Data Preprocessing

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text, stop_words, lemmatizer):
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize
    tokens = text.split()
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

def preprocess_with_nltk(data):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    tqdm.pandas(desc="Preprocessing text")
    data['review'] = data['review'].progress_apply(lambda x: preprocess_text(x, stop_words, lemmatizer))
    return data

train_data = preprocess_with_nltk(train_data)
test_data = preprocess_with_nltk(test_data)

Let's compare the data to what we had before preprocessing

In [ ]:
print(train_data.head())

### We will be using both TF-IDF and Bag of Words for feature extraction, and compare performance between the two.

In [ ]:
def extract_tfidf_features(train_data, test_data, max_features=5000):
    print("Extracting TF-IDF features...")
    tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
    X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['review'])
    X_test_tfidf = tfidf_vectorizer.transform(test_data['review'])
    return X_train_tfidf, X_test_tfidf, tfidf_vectorizer

def extract_bow_features(train_data, test_data, max_features=5000):
    print("Extracting Bag of Words features...")
    bow_vectorizer = CountVectorizer(max_features=max_features)
    X_train_bow = bow_vectorizer.fit_transform(train_data['review'])
    X_test_bow = bow_vectorizer.transform(test_data['review'])
    return X_train_bow, X_test_bow, bow_vectorizer

X_train_bow, X_test_bow, bow_vectorizer = extract_bow_features(train_data, test_data)
X_train_tfidf, X_test_tfidf, tfidf_vectorizer = extract_tfidf_features(train_data, test_data)

# Labels
y_train = train_data['sentiment']
y_test = test_data['sentiment']

In [ ]:
# Compare dimensions and sparsity
print("Bag of Words Feature Matrix:")
print(f"Shape: {X_train_bow.shape}")
print("Sparsity (BoW): {:.2f}%".format(100 * (X_train_bow.nnz / (X_train_bow.shape[0] * X_train_bow.shape[1]))))

print("\nTF-IDF Feature Matrix:")
print(f"Shape: {X_train_tfidf.shape}")
print("Sparsity (TF-IDF): {:.2f}%".format(100 * (X_train_tfidf.nnz / (X_train_tfidf.shape[0] * X_train_tfidf.shape[1]))))

### Naive Bayes Model Training on both TF-IDF and BoW feature sets

In [ ]:
def train_naive_bayes(X_train, X_test, y_train, y_test, feature_name):
    print(f"\nTraining Naïve Bayes on {feature_name}...")
    model = MultinomialNB()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluation
    print(f"Results for {feature_name}:")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    return model

# Train NB on BoW feature set
nb_bow_model = train_naive_bayes(X_train_bow, X_test_bow, y_train, y_test, "Bag of Words")

# Train NB on TF-IDF feature set
nb_tfidf_model = train_naive_bayes(X_train_tfidf, X_test_tfidf, y_train, y_test, "TF-IDF")  

### Train RNN (LSTM) on both TF-IDF and BoW Feature Sets

In [ ]:
# Tokenization and Padding
def preprocess_for_lstm(train_data, test_data, max_features=5000, max_length=500):
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(train_data['review'])

    # Convert text to sequences
    X_train_seq = tokenizer.texts_to_sequences(train_data['review'])
    X_test_seq = tokenizer.texts_to_sequences(test_data['review'])

    # Pad sequences to ensure uniform length
    X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
    X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

    return X_train_padded, X_test_padded, tokenizer

# Preprocess the data
max_features = 5000  # Number of unique words to consider
max_length = 500  # Maximum length of sequences
X_train_padded, X_test_padded, tokenizer = preprocess_for_lstm(train_data, test_data, max_features, max_length)

# Labels
y_train = train_data['sentiment']
y_test = test_data['sentiment']

def build_lstm_model(max_features):
    model = Sequential([
        Embedding(input_dim=max_features, output_dim=128),
        LSTM(64, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_lstm_model(max_features, max_length)

# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

print("Training LSTM...")
history = model.fit(
    X_train_padded, y_train,
    batch_size=32,
    epochs=10,
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluation
print("\nEvaluating on test data...")
y_pred = (model.predict(X_test_padded) > 0.5).astype("int32")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")